In [1]:
import os 
import xarray as xr
import numpy as np
import glob
import pandas as pd
from sklearn.cluster import KMeans, AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler, normalize
import itertools
from numpy.lib.stride_tricks import sliding_window_view
import datetime
from matplotlib import pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.ticker as mticker
from cartopy.mpl.ticker import (LongitudeFormatter, LatitudeFormatter)


In [131]:
project_dir = '/Users/huripari/Documents/PhD/TCs_Genesis'
data_dir = os.path.join(project_dir, 'data')

dataset = 'ERA5dv'
resolution = '2.5x2.5'
vars_group = 'SSTA'
year = 2019
var = 'ssta20'
path_predictor = os.path.join(data_dir, dataset, resolution, vars_group, vars_group)
path = path_predictor + f'_{resolution}_{year}.nc'
train_data = xr.open_dataset(path)[var]

In [154]:
first_year = 1980
last_year = 2021
years = np.arange(first_year, last_year+1)

tcg_ds = xr.concat([xr.open_dataset(os.path.join(data_dir, 'IBTrACS', resolution, 'TCG', f'TCG_{resolution}_{year}.nc')) for year in years], dim='time')
tcg_ds

<xarray.Dataset>
Dimensions:    (time: 504, latitude: 33, longitude: 144)
Coordinates:
  * longitude  (longitude) float32 -180.0 -177.5 -175.0 ... 172.5 175.0 177.5
  * latitude   (latitude) float32 40.0 37.5 35.0 32.5 ... -35.0 -37.5 -40.0
  * time       (time) datetime64[ns] 1980-01-01 1980-02-01 ... 2021-12-01
Data variables:
    tcg        (time, latitude, longitude) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0

In [155]:
tcg_ds.sum(dim=['latitude', 'longitude'])

<xarray.Dataset>
Dimensions:  (time: 504)
Coordinates:
  * time     (time) datetime64[ns] 1980-01-01 1980-02-01 ... 2021-12-01
Data variables:
    tcg      (time) float64 6.0 5.0 5.0 2.0 4.0 4.0 ... 16.0 15.0 7.0 5.0 4.0

In [156]:
tcg_ds.tcg.sum(dim=['latitude', 'longitude']).values

array([ 6.,  5.,  5.,  2.,  4.,  4.,  9.,  7., 13.,  7.,  5.,  5.,  3.,
        7.,  6.,  3.,  3.,  5.,  7., 14., 11.,  7.,  6.,  6.,  7.,  6.,
        5.,  3.,  4.,  7.,  9., 12., 12.,  9.,  5.,  4.,  5.,  5.,  4.,
        5.,  1.,  2.,  9., 10.,  8., 11.,  8.,  8.,  7., 10.,  6.,  2.,
        2.,  5.,  8., 15., 15., 10.,  8.,  7., 10.,  8.,  6.,  3.,  3.,
        8., 10., 15., 13., 11.,  4.,  4.,  9.,  9.,  6.,  4.,  5.,  6.,
        7.,  9., 10.,  6.,  7.,  6.,  6.,  9.,  2.,  4.,  1.,  4., 10.,
       13., 12.,  8.,  3.,  2.,  6.,  5.,  2.,  1.,  3.,  5.,  8., 15.,
       16.,  6.,  7.,  4.,  7.,  9.,  5.,  5.,  5.,  6., 14., 15., 12.,
        5.,  5.,  3.,  6.,  3.,  9.,  3.,  4.,  7., 10., 15., 11., 11.,
        5.,  4.,  1.,  6.,  4.,  4.,  3.,  6.,  7.,  8., 12.,  8., 10.,
        6.,  4., 10.,  4.,  4.,  1.,  6., 10., 14., 16., 16.,  5.,  4.,
        4.,  8.,  2.,  3.,  2.,  4.,  7., 16., 12.,  5.,  5.,  7.,  8.,
        3.,  8.,  4.,  1.,  6., 12., 14., 16.,  9.,  5.,  3.,  5